# **Final year project - Prototype**

## Introduction


For the first step to run this notebook, you will need to upload the API token that I have generated in Kaggle.

Please, run the following block of code: 

In [60]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"osodracd","key":"33b1c744630e7b5e17c75e86a314d8c3"}'}

Next click on *Choose files*. This will prompt a upload window where you need to locate, select and upload the kaggle.json file that I have attached to the zip file. 

Once the file is visible in the folder tree on the left side, you can follow to the next block of code. 

## The dataset 

The data consists of 48x48 pixel grayscale images of faces. The faces have been automatically registered so that the face is more or less centred and occupies about the same amount of space in each image.

The task is to categorize each face based on the emotion shown in the facial expression into one of seven categories (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral). The training set consists of 28,709 examples and the public test set consists of 3,589 examples.

In [ ]:
# Kaggle installation
! pip install kaggle
# Create Kaggle directory
! mkdir ~/.kaggle
# Copy the kaggle.json file into the current directory
! cp kaggle.json ~/.kaggle/
# provide owner full read and write access to the file
! chmod 600 ~/.kaggle/kaggle.json
# Download the dataset zip file
! kaggle datasets download msambare/fer2013
# Extract the files withtin the zip file
! unzip fer2013.zip
print('Unzip completed.')

Now that the dataset is downloaded and accessible, I will import all the necessary libraries and modules

In [ ]:
# Import statements
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/
import seaborn as sns
import tensorflow as tf
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import matplotlib.pyplot as plt

from keras_preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization,Activation, MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import adam_v2, rmsprop_v2

## Preparing the data

In [ ]:
# Standardized size of images
picture_size = 48
folder_path = "/content/"

In [ ]:
# Plotting neutral images from the folder
expression = "sad"
# Defining mapplotlib graph
plt.figure(figsize=(12, 12))
# Plotting first 9 images
for i in range (1, 10, 1):
    plt.subplot(3, 3, i)
    img = load_img(folder_path+"train/"+expression+"/" + os.listdir(folder_path+"train/"+expression)[i],target_size= (picture_size, picture_size))
    plt.imshow(img)
plt.show()
print('Example of ' + expression + ' faces.')

In [ ]:
# Test plotting for a different expression
expression = "happy"
# Defining mapplotlib graph
plt.figure(figsize=(12, 12))
# Plotting first 9 images
for i in range (1, 10, 1):
    plt.subplot(3, 3, i)
    img = load_img(folder_path+"train/"+expression+"/" + os.listdir(folder_path+"train/"+expression)[i],target_size= (picture_size, picture_size))
    plt.imshow(img)
plt.show()
print('Example of ' + expression + ' faces.')

## Training and validation

In [51]:
# Training examples used per iteration
batch_size = 128

# Defining variables as image data generator
datagenerator_train = ImageDataGenerator()
datagenerator_val = ImageDataGenerator()

# Variables containers of the data from each directory
## Will collect all the images present in the directory and stored them at the variable
train_set = datagenerator_train.flow_from_directory(folder_path + "train",
                                                    target_size = (picture_size, picture_size),
                                                    color_mode = "grayscale",
                                                    batch_size = batch_size,
                                                    class_mode = "categorical",
                                                    shuffle = True)

# Variables containers of the data from each directory
## Will collect all the images present in the directory and stored them at the variable
test_set = datagenerator_train.flow_from_directory(folder_path+"test",
                                                    target_size = (picture_size, picture_size),
                                                    color_mode = "grayscale",
                                                    batch_size = batch_size,
                                                    class_mode = "categorical",
                                                    shuffle = True)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


## Initial Model

In this section I will build the initial model.

In [56]:
number_of_classes = 7

model = Sequential()
# CNN layer 1
model.add(Conv2D(32,(3,3), padding='same', input_shape = (48, 48, 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25)) # Prevention of overfitting. Drops 25% of the data, setting those datapoints to 0. Generalize better and reach higher accurarcy

# CNN layer 2
model.add(Conv2D(64,(3,3), padding='same', input_shape = (48, 48, 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25)) # Prevention of overfitting

# CNN layer 3
model.add(Conv2D(128,(3,3), padding='same', input_shape = (48, 48, 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25)) # Prevention of overfitting

# CNN layer 4
model.add(Conv2D(512,(3,3), padding='same', input_shape = (48, 48, 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25)) # Prevention of overfitting

model.add(Flatten()) # Collapse input size to one dimensional array

# Fully connected layer 1
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# Fully connected layer 2
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(number_of_classes, activation='softmax'))

opt = Adam (learning_rate=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics = ['accuracy'])
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 48, 48, 32)        320       
                                                                 
 batch_normalization_18 (Bat  (None, 48, 48, 32)       128       
 chNormalization)                                                
                                                                 
 activation_18 (Activation)  (None, 48, 48, 32)        0         
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 24, 24, 32)       0         
 g2D)                                                            
                                                                 
 dropout_18 (Dropout)        (None, 24, 24, 32)        0         
                                                                 
 conv2d_13 (Conv2D)          (None, 24, 24, 64)       

In [59]:
epochs = 10
history = model.fit_generator (generator=train_set,
                               steps_per_epoch=train_set.n//train_set.batch_size, # floor division of train set by batch size
                               epochs = epochs,
                               validation_data = test_set,
                               validation_steps = test_set.n//test_set.batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/10
224/224 [==============================] - 335s 1s/step - loss: 1.6545 - accuracy: 0.3572 - val_loss: 1.6251 - val_accuracy: 0.3790
Epoch 2/10
224/224 [==============================] - 338s 2s/step - loss: 1.6140 - accuracy: 0.3756 - val_loss: 1.6132 - val_accuracy: 0.3945
Epoch 3/10
224/224 [==============================] - 339s 2s/step - loss: 1.5812 - accuracy: 0.3878 - val_loss: 1.5828 - val_accuracy: 0.4008
Epoch 4/10
224/224 [==============================] - 337s 2s/step - loss: 1.5478 - accuracy: 0.4028 - val_loss: 1.5618 - val_accuracy: 0.4051
Epoch 5/10
224/224 [==============================] - 335s 1s/step - loss: 1.5159 - accuracy: 0.4134 - val_loss: 1.5117 - val_accuracy: 0.4261
Epoch 6/10
224/224 [==============================] - 335s 1s/step - loss: 1.4886 - accuracy: 0.4251 - val_loss: 1.5034 - val_accuracy: 0.4340
Epoch 7/10
224/224 [==============================] - 338s 2s/step - loss: 1.4563 - accuracy: 0.4385 - val_loss: 1.4773 - val_accuracy: 0.4443

## Conclusion


For this project specifications I have set the aim to achieve 66% of effectiveness, in line similar studies. 

Overall, this prototype would have a baseline of approximately 15% accuracy for a balanced dataset, taking that 100% accuracy would be divided by the 7 classes. The current results prove that this prototype has statistical power for it has achieved validation accuracy higher than 15%, at 47+%, hence providing the required proof of concept. 

Moving forward, I will improve the model through the tuning of hyperparameters such as the optimizer, the learning rate, the number of layers, the number of units per layer and the momentum. I will also fight overfitting through the application of measures such as adding more data, network size reduction and weight regularization.

Once the deep learning model has achieved accuracy close to the minimum viable product (MVP), I will expand it into the final project by importing module CV2 and connect the notebook to a live webcam with the purpose of feeding unseen data, ultimately testing the effectiveness and accomplishing the planned Convolutional Neural Networks for Emotion Classification. Then, I will develop the research aspect of the project through the execution of the proposed user testing, culminating in a comparation and evaluation study.

I am confident that this prototype implements all the functionality of the project to an initially working standard, being very clear that the project is doable within the timeframe of this module, for is already close to the MVP, which makes me believe that is possible to produce a complete and finished product as idealized initially.